In [25]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [26]:
df_raw = pd.read_table("video games sales.csv", sep=",")
df_raw.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [27]:
df_raw.isna().sum() / len(df_raw) # процент пропусков по столбцам

Rank            0.000000
Name            0.000000
Platform        0.000000
Year            0.016327
Genre           0.000000
Publisher       0.003494
NA_Sales        0.000000
EU_Sales        0.000000
JP_Sales        0.000000
Other_Sales     0.000000
Global_Sales    0.000000
dtype: float64

In [28]:
most_common_publisher = df_raw["Publisher"].mode()[0]
df_publisher_filled = df_raw["Publisher"].fillna(most_common_publisher)

In [29]:
df_clean = df_raw.copy()
df_clean["Publisher"] = df_publisher_filled
df_clean.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [30]:
df_raw['Year'] = pd.to_numeric(df_raw['Year'], errors='coerce')

df_model = df_raw.dropna(subset=['Year']).copy()

features = ['Platform', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
df_model = df_model[features + ['Year']]
df_model[['Platform', 'Genre', 'Publisher']] = df_model[['Platform', 'Genre', 'Publisher']].fillna('Unknown')

df_encoded = pd.get_dummies(df_model, columns=['Platform', 'Genre', 'Publisher'])

X = df_encoded.drop('Year', axis=1)
y = df_encoded['Year']

In [31]:
pipe = Pipeline(steps=[
    ('imputer', KNNImputer(
        n_neighbors=5,
        weights='distance',
        add_indicator=False)),
    ('scaler', StandardScaler()),
    ('regressor', Lasso(max_iter=2000))
])

param_grid = {
    'imputer__n_neighbors': [3, 5, 10],
    'imputer__weights': ['uniform', 'distance'],
    'imputer__add_indicator': [True, False],
    'regressor__alpha': [10, 100, 200]
}

grid_search = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1, scoring='r2')
grid_search.fit(X, y)

print("Лучшие параметры:\n", grid_search.best_params_)

Лучшие параметры:
 {'imputer__add_indicator': True, 'imputer__n_neighbors': 3, 'imputer__weights': 'uniform', 'regressor__alpha': 10}


In [32]:
best_imputer = KNNImputer(
    n_neighbors=grid_search.best_params_['imputer__n_neighbors'],
    weights=grid_search.best_params_['imputer__weights'],
    add_indicator=grid_search.best_params_['imputer__add_indicator'],
    metric='nan_euclidean'
)

df_all = df_raw[features + ['Year']].copy()
df_all[['Platform', 'Genre', 'Publisher']] = df_all[['Platform', 'Genre', 'Publisher']].fillna('Unknown')
df_all_encoded = pd.get_dummies(df_all, columns=['Platform', 'Genre', 'Publisher'])

imputed_array = best_imputer.fit_transform(df_all_encoded)
original_cols = df_all_encoded.columns.tolist()

indicator_cols = [f"{col}_missing" for col in df_all_encoded.columns if df_all_encoded[col].isnull().any()]

all_cols = original_cols + indicator_cols

df_imputed = pd.DataFrame(imputed_array, columns=all_cols)

df_clean['Year'] = df_imputed['Year'].round().astype(int)


In [33]:
df_clean.isna().sum() / len(df_raw) # процент пропусков по столбцам

Rank            0.0
Name            0.0
Platform        0.0
Year            0.0
Genre           0.0
Publisher       0.0
NA_Sales        0.0
EU_Sales        0.0
JP_Sales        0.0
Other_Sales     0.0
Global_Sales    0.0
dtype: float64

In [34]:
categorical_cols = ['Genre']

df_raw_enc = df_raw.copy()
df_raw_enc[categorical_cols] = df_raw_enc[categorical_cols].fillna('Unknown')

df_encoded = pd.get_dummies(df_raw_enc, columns=categorical_cols)

print("Форма до кодирования:", df_raw_enc.shape)
print("Форма после кодирования:", df_encoded.shape)
df_encoded.head()

Форма до кодирования: (16598, 11)
Форма после кодирования: (16598, 22)


,Rank,Name,Platform,Year,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,Genre_Fighting,Genre_Misc,Genre_Platform,Genre_Puzzle,Genre_Racing,Genre_Role-Playing,Genre_Shooter,Genre_Simulation,Genre_Sports,Genre_Strategy
0,1,Wii Sports,Wii,2006.0,Nintendo,41.49,29.02,3.77,8.46,82.74,...,False,False,False,False,False,False,False,False,True,False
1,2,Super Mario Bros.,NES,1985.0,Nintendo,29.08,3.58,6.81,0.77,40.24,...,False,False,True,False,False,False,False,False,False,False
2,3,Mario Kart Wii,Wii,2008.0,Nintendo,15.85,12.88,3.79,3.31,35.82,...,False,False,False,False,True,False,False,False,False,False
3,4,Wii Sports Resort,Wii,2009.0,Nintendo,15.75,11.01,3.28,2.96,33.00,...,False,False,False,False,False,False,False,False,True,False
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Nintendo,11.27,8.89,10.22,1.00,31.37,...,False,False,False,False,False,True,False,False,False,False


In [35]:
df_raw_normalized = df_raw.copy()

df_raw_normalized['Year'] = pd.to_numeric(df_raw_normalized['Year'], errors='coerce')

numeric_cols = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']

df_raw_normalized[numeric_cols] = df_raw_normalized[numeric_cols].fillna(df_raw_normalized[numeric_cols].median())

scaler = StandardScaler()

df_scaled = df_raw_normalized.copy()
df_scaled[numeric_cols] = scaler.fit_transform(df_raw_normalized[numeric_cols])

print("Среднее после нормализации (должно быть ~0):\n", df_scaled[numeric_cols].mean())
print("Ст. отклонение (должно быть ~1):\n", df_scaled[numeric_cols].std())
df_scaled.head()


Среднее после нормализации (должно быть ~0):
 NA_Sales       -4.452129e-17
EU_Sales        7.191901e-17
JP_Sales       -7.876844e-17
Other_Sales    -6.164487e-17
Global_Sales   -4.623365e-17
dtype: float64
Ст. отклонение (должно быть ~1):
 NA_Sales        1.00003
EU_Sales        1.00003
JP_Sales        1.00003
Other_Sales     1.00003
Global_Sales    1.00003
dtype: float64


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,50.480508,57.136930,11.938058,44.606085,52.864025
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,35.284437,6.794188,21.767296,3.828224,25.532503
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,19.084273,25.197785,12.002724,17.297115,22.690025
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,18.961823,21.497277,10.353740,15.441165,20.876498
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,13.476053,17.302048,32.792857,5.047848,19.828254
